In [1]:
import pyspark as ps
from collections import Counter
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import random
from data_prep import parse_line
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )


sc = spark.sparkContext

col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    cols[col_names[i]] = i 
    
def casting_function(row):
    if row[0] == 'marketplace':
        return ()
    if len(row)!=15:
        return(len(row))
    (marketplace, customer_id, review_id, product_id, product_parent, product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date) = row
    
    
    return (marketplace, str(customer_id), str(review_id), str(product_id), product_parent, str(product_title),str(product_category),int(star_rating),int(helpful_votes),int(total_votes),vine,verified_purchase,str(review_headline),str(review_body),review_date)

def time_cast(row):
    pattern = '%Y-%m-%d'
    row =list(row)
    row[14] = int(time.mktime(time.strptime(row[14], pattern)))/86400
    return tuple(row)

def toCSVLine(data):
  return ','.join(str(d) for d in data)

def make_parse(row):
    row = list(row)
    row.append(parse_line(row[13]))
    return tuple(row)

file = open("data/S3_Repos.txt","r+")
repos = []
for line in file.readlines():
    line = line.replace('https://s3.amazonaws.com/amazon-reviews-pds/tsv/','')
    line = line.replace('\n','')
    repos.append('s3://amazon-reviews-pds/tsv/'+line)
     

total_row_counts = []
final_rows = []
parsed_final = []
opt_rows = []
Scale = []
Names = []
tenK_sizes = []
loop_num = 0

repos = [repos[1]]


for repo in repos:
    loop_num = loop_num + 1
    print(str(loop_num)+'-Begin: '+repo.replace('s3://amazon-reviews-pds/tsv/amazon_reviews_','').replace('.tsv.gz',''))
    rdd_books = sc.textFile(repo).map(lambda rowstr : rowstr.split("\t")).map(casting_function)
    rdd_books =rdd_books.filter(lambda x: len(x)==15).map(time_cast)
    rdd_sample = rdd_books.filter(lambda x: x[7]==1).filter(lambda x: x[9]>5)
    print('Check_1')
    tiny_df = pd.DataFrame(rdd_sample.take(10000))
    print('Check_2')
    tiny_df.to_csv('data/size_check.csv')
    tenK_size = os.path.getsize('data/size_check.csv')
    print('Check_3')
    total_rows = rdd_sample.count()
    total_row_counts.append(total_rows)
    print('Check_4')
    optimal_bytes = 90000000
    
    tenK_scale = optimal_bytes/tenK_size
    tenK_sizes.append(tenK_size)
    
    optimal_rows = 10000*tenK_scale
    opt_rows.append(optimal_rows)
    file_name = 'data/Spark_Pulls/'+repo.replace('s3://amazon-reviews-pds/tsv/amazon_reviews_','').replace('.tsv.gz','')+'.csv'
    print('Check_5')
    if total_rows <optimal_rows:
        print('Check_6a')
        temp = pd.DataFrame(rdd_sample.map(make_parse).collect())
        print('Check_7a')
        temp.to_csv(file_name)
        Scale.append(1)
        final_rows.append(total_rows)
    else:
        scale = optimal_rows/total_rows
        Scale.append(scale)
        final_rows.append(total_rows*scale)
        rdd_sample = rdd_sample.filter(lambda x: random.random() < scale).map(make_parse)
        print('Check_6b')
        temp = pd.DataFrame(rdd_sample.collect())
        print('Check_6b')
        temp.to_csv(file_name)
        print('Check_7b')
    optimal_bytes = 95000000
    parsed_size = os.path.getsize(file_name)
    if parsed_size > optimal_bytes:
        temp = temp.sample(frac=optimal_bytes/parsed_size, random_state=1)
        temp.to_csv(file_name)
    parsed_final.append(len(temp))
    print('finished: '+repo.replace('s3://amazon-reviews-pds/tsv/amazon_reviews_','').replace('.tsv.gz',''))
    Names.append(repo.replace('s3://amazon-reviews-pds/tsv/amazon_reviews_','').replace('.tsv.gz',''))
df = pd.DataFrame()
df['Name'] = Names
df['Scale'] = Scale
df['total_rows']  =total_row_counts
df['final_rows'] = final_rows
df['parsed_final'] = parsed_final
df['opt_rows'] = opt_rows
df['tenK_size'] = tenK_sizes
df.to_csv('data/spark_metrics.csv')
    

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/ec2-user/nltk_data...
[nltk_data]    |   Package movie_reviews is a

1-Begin: us_Watches_v1_00
Check_1
Check_2
Check_3
Check_4
Check_5
Check_6a
Check_7a
finished: us_Watches_v1_00


In [2]:
print('COWSSSSS')

COWSSSSS


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,US,18949762,R12U0031MS8AZU,B0122MC0BA,267006201,LNI Fashion Stainless Steel Band Watch Mk,Watches,1,6,8,N,Y,Four Stars,"When I first saw the watch, I was so happy not...",16678.0,first saw watch happy discover watch work disa...
1,US,21005490,R3JFUP4ZFPHLZ8,B002SSUQFG,354933056,Seiko Men's SNK809 Seiko 5 Automatic Stainless...,Watches,1,0,6,N,N,This watch will not allow for second hand sett...,This watch will not allow for second hand sett...,16678.0,watch allow second hand set two watch send cou...
2,US,45416451,R29WC4B3ACC3RZ,B004JY0KP8,815774729,Seiko Men's SNDC31 Classic Stainless Steel Chr...,Watches,1,2,8,N,Y,One Star,Broke in two weeks,16678.0,broke two week
3,US,14870011,R6F9VY91ADPLA,B002ZLE41I,118278818,Grand Seiko Wristwatch Sbgx061 Mens,Watches,1,8,30,N,N,NOT a GS,NOT a Grand Seiko - do not buy at this price.<...,16678.0,grand seiko buy br br know poster realizes wat...
4,US,6402475,RONNVK5TAN4U6,B00KMAKALC,600186626,Seiko Men's SNE335 Dress Solar Analog Display ...,Watches,1,1,6,N,Y,One Star,To big and bulky for a small arm. RETURNED,16676.0,big bulky small arm return
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11397,US,22332281,R2QYL631P4ORX5,B00005QEME,252821780,Men's Timex Internet Messenger Sport Watch,Watches,1,0,24,N,N,gay,i have a lot of brothers and they like really ...,12404.0,lot brother like really anything watch watch l...
11398,US,36950265,R1LB90VOMIYWEG,B0000UIWY6,236124170,Disney Kids' MC0292 Princess Trio Flower Charm...,Watches,1,8,10,N,N,No Learning Value,Although I thought this watch looks adorable f...,12385.0,although think watch look adorable princess mi...
11399,US,38105554,R1721LXOWD6H3C,B00005OT8Z,122254165,Timex Ironman Triathlon Digital Heart Rate Mon...,Watches,1,11,27,N,N,I was excited,I was very excited about this product when I p...,11948.0,excited product place order think would help w...
11400,US,49987382,RPAY64ZST7TE3,B00005OT91,458246193,Timex Men's T44661 Expedition Metal Tech Watch,Watches,1,10,11,N,N,Piece of [stuff],It is suppossed to take a lickin.. BUT IT DOES...,11822.0,suppossed take back fell ski back ame along ba...


In [4]:
df

,Name,Scale,total_rows,final_rows,parsed_final,opt_rows,tenK_size
0,us_Watches_v1_00,1,11402,11402,11402,131685.219651,6834480
